In [1]:
import db_adapter as db
import pandas as pd

In [2]:
conn = db.Connection()

In [3]:
tables = conn.get_tables()

In [4]:
for name, table in tables.items():
    print(name)

Clientes
Fornecedores
Fornecedores_Produtos
Lojas
Produtos
Vendas
Vendas_Produtos


In [5]:
dfs = {name: pd.DataFrame(table.scan()) for name, table in tables.items()}

In [6]:
dfs['Vendas_Produtos']

,venda,preco,quantidade,produto
0,3,50,50,0
1,3,20,10,2
2,2,600,1,5
3,1,250,1,1
4,1,250,1,4
5,0,6,15,3


In [7]:
dfs['Produtos']

,codigo,nome,descricao
0,3,Pepino,Pepino!
1,2,Água em Pó,Basta colocar água!
2,4,Mineirinho Adventures,Nada a comentar!
3,1,Vaporizador RGB,Infinitas opções de customização!
4,0,Supositório Gamer,Cague como um campeão!
5,5,Dick Augmentator Tabajara,Aumente seu pepino agora mesmo!


In [8]:
dfs['Vendas']

,loja,num_nota_fiscal,cliente,data
0,3,1,3,2023-08-18
1,2,3,4,2023-01-20
2,1,2,2,2023-03-03
3,0,0,1,2023-06-14


In [9]:
dfs['Clientes']

,endereco,codigo,nome
0,Fortaleza,3,Roberto
1,Recife,2,Carlos
2,Porto Alegre,4,Maria
3,São Paulo,1,Antônio
4,Salvador,0,Raíssa


#### Analyzing the sales for Maria

In [18]:
tables['Clientes'].key_schema()

[{'AttributeName': 'codigo', 'KeyType': 'HASH'}]

> Note that if *nome* was the partition key, the scan below would not be necessary. This was a database design mistake on my part.

In [17]:

maria_id = tables['Clientes'].scan(db.Attr('nome').eq('Maria'))[0]['codigo']
maria_id

Decimal('4')

In [25]:
sale = tables['Vendas'].scan(db.Attr('cliente').eq(maria_id))[0]
sale_id = sale['num_nota_fiscal']
sale_id

Decimal('3')

In [27]:
maria_sale = tables['Vendas_Produtos'].query(db.Key('venda').eq(sale_id))
maria_sale = pd.DataFrame(maria_sale)
maria_sale

,venda,preco,quantidade,produto
0,3,50,50,0
1,3,20,10,2
